In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset
from bpemb import BPEmb
from datasets import load_dataset

In [7]:
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")
device

device(type='cuda')

In [8]:
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"]

Using custom data configuration copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6
Reusing dataset parquet (/home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 116067
    })
    validation: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 13325
    })
})

In [10]:
validation_set = dataset["validation"]

In [11]:
def getLanguageDataSet(data, language):
    def printAndL(x):
        return x["language"] == language
    return data.filter(printAndL)

def getEnglishDataSet(data):
    return getLanguageDataSet(data, "english")

# keep only english data
english_train_set = getEnglishDataSet(train_set)
english_validation_set = getEnglishDataSet(validation_set)

Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-dbe296a96306378b.arrow
Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-9c5553adc4f99418.arrow


In [12]:
# delete useless data
english_train_set = english_train_set.remove_columns(["document_title", "language", "document_url"])
english_validation_set = english_validation_set.remove_columns(["document_title", "language", "document_url"])

In [13]:
english_train_set

Dataset({
    features: ['question_text', 'annotations', 'document_plaintext'],
    num_rows: 7389
})

In [14]:
def label_map_func(examples):
    labels = []
    for i in examples["annotations"]:
        if i["answer_start"] == [-1]:
            labels.append([0])

        else:
            labels.append([1])
    return {"label": labels}

english_label_train_set = english_train_set.map(label_map_func , batched=True, num_proc=4, remove_columns=["annotations"])

Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-7804682ce4cfa673.arrow


Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-73c4a07fb746d0f8.arrow


Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-ca751636e6333f5b.arrow


Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e42d7860fb83f452.arrow


In [15]:
english_label_train_set["document_plaintext"][0]

'Quantum field theory naturally began with the study of electromagnetic interactions, as the electromagnetic field was the only known classical field as of the 1920s.[8]:1'

In [16]:
import transformers
from transformers import AutoTokenizer
transformer_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

ModuleNotFoundError: No module named 'transformers'

In [ ]:
block_size = 128

def document_tokenize_function(examples):
    return transformer_tokenizer(examples['document_plaintext'], padding="max_length")

def question_tokenize_function(examples):
    return transformer_tokenizer(examples['question_text'], padding="max_length")


In [ ]:
document_unsupervised_tok = english_label_train_set.map(document_tokenize_function, batched=True, num_proc=4, remove_columns=["document_plaintext"])
document_unsupervised_tok

In [ ]:

def switch_name1(examples):
    return {"document_input_ids": examples['input_ids']}
document_unsupervised_tok = document_unsupervised_tok.map(switch_name1, batched=True, num_proc=4, remove_columns=["input_ids"])


In [ ]:
def switch_name2(examples):
    return {"document_attention_mask": examples['attention_mask']}
document_unsupervised_tok = document_unsupervised_tok.map(switch_name2, batched=True, num_proc=4, remove_columns=["attention_mask"])

In [ ]:
document_unsupervised_tok

In [ ]:
question_and_document_unsupervised_tok = document_unsupervised_tok.map(question_tokenize_function, batched=True, num_proc=4, remove_columns=["question_text"])

In [ ]:
def switch_name1(examples):
    return {"question_input_ids": examples['input_ids']}
question_and_document_unsupervised_tok = question_and_document_unsupervised_tok.map(switch_name1, batched=True, num_proc=4, remove_columns=["input_ids"])


In [ ]:

def switch_name2(examples):
    return {"question_attention_mask": examples['attention_mask']}
question_and_document_unsupervised_tok = question_and_document_unsupervised_tok.map(switch_name2, batched=True, num_proc=4, remove_columns=["attention_mask"])

In [ ]:
question_and_document_unsupervised_tok

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
'''
@File    : preprocess.py.py
@IDE     : PyCharm
@Author  : Yaokun Li
@Date    : 2022/10/20 15:48
@Description :
'''
import torch
import numpy as np
from torch.utils.data import Dataset
from bpemb import BPEmb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer

device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")


dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"]
validation_set = dataset["validation"]

def getLanguageDataSet(data, language):
    def printAndL(x):
        return x["language"] == language
    return data.filter(printAndL)

def getEnglishDataSet(data):
    return getLanguageDataSet(data, "english")

# keep only english data
english_train_set = getEnglishDataSet(train_set)
english_validation_set = getEnglishDataSet(validation_set)

# delete useless data
english_train_set = english_train_set.remove_columns(["document_title", "language", "document_url"])
english_validation_set = english_validation_set.remove_columns(["document_title", "language", "document_url"])

def label_map_func(examples):
    labels = []
    for i in examples["annotations"]:
        if i["answer_start"] == [-1]:
            labels.append([0])

        else:
            labels.append([1])
    return {"label": labels}

english_label_train_set = english_train_set.map(label_map_func , batched=True, num_proc=4, remove_columns=["annotations"])
english_label_val_set = english_validation_set.map(label_map_func , batched=True, num_proc=4, remove_columns=["annotations"])

transformer_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

block_size = 128

def document_tokenize_function(examples):
    return transformer_tokenizer(examples['document_plaintext'], padding="max_length")

def question_tokenenglish_label_train_setize_function(examples):
    return transformer_tokenizer(examples['question_text'], padding="max_length")

def switch_doc_name1(examples):
    return {"document_input_ids": examples['input_ids']}

def switch_doc_name2(examples):
    return {"document_attention_mask": examples['attention_mask']}

def switch_ques_name1(examples):
    return {"question_input_ids": examples['input_ids']}

def switch_ques_name2(examples):
    return {"question_attention_mask": examples['attention_mask']}

def get_final_dataset(data_set):
    return data_set\
        .map(document_tokenize_function, batched=True, num_proc=4, remove_columns=["document_plaintext"])\
        .map(switch_doc_name1, batched=True, num_proc=4, remove_columns=["input_ids"])\
        .map(switch_doc_name2, batched=True, num_proc=4, remove_columns=["attention_mask"])\
        .map(question_tokenize_function, batched=True, num_proc=4, remove_columns=["question_text"])\
        .map(switch_ques_name1, batched=True, num_proc=4, remove_columns=["input_ids"])\
        .map(switch_ques_name2, batched=True, num_proc=4, remove_columns=["attention_mask"])


train_dl = get_final_dataset(english_label_train_set).remove_columns("token_type_ids")
val_dl = get_final_dataset(english_label_val_set).remove_columns("token_type_ids")

In [ ]:
def collate_batch_bilstm(dataset):
    print(dataset)
    label = torch.Tensor(dataset["label"]).cuda()
    document = torch.Tensor(dataset["document_input_ids"]).cuda()
    document_mask = torch.tensor(dataset["document_attention_mask"]).cuda()
    question = torch.tensor(dataset["question_input_ids"]).cuda()
    question_mask = torch.tensor(dataset["question_attention_mask"]).cuda()

    return document, document_mask, question, question_mask, label


In [ ]:
train_dl = torch.utils.data.DataLoader(train_dl, batch_size=32, collate_fn=collate_batch_bilstm)
valid_dl = torch.utils.data.DataLoader(val_dl, batch_size=32, collate_fn=collate_batch_bilstm)

In [286]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
'''
@File    : preprocess.py.py
@IDE     : PyCharm
@Author  : Yaokun Li
@Date    : 2022/10/20 15:48
@Description :
'''
import torch
import numpy as np
from torch.utils.data import Dataset
from bpemb import BPEmb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer

device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")


dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"]
validation_set = dataset["validation"]

def getLanguageDataSet(data, language):
    def printAndL(x):
        return x["language"] == language
    return data.filter(printAndL)

def getEnglishDataSet(data):
    return getLanguageDataSet(data, "english")

# keep only english data
english_train_set = getEnglishDataSet(train_set)
english_validation_set = getEnglishDataSet(validation_set)

# delete useless data
english_train_set = english_train_set.remove_columns(["document_title", "language", "document_url"])
english_validation_set = english_validation_set.remove_columns(["document_title", "language", "document_url"])

def label_map_func(examples):
    labels = []
    for i in examples["annotations"]:
        if i["answer_start"] == [-1]:
            labels.append([0])

        else:
            labels.append([1])
    return {"label": labels}

english_label_train_set = english_train_set.map(label_map_func , batched=True, num_proc=4, remove_columns=["annotations"])
english_label_val_set = english_validation_set.map(label_map_func , batched=True, num_proc=4, remove_columns=["annotations"])

transformer_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

block_size = 128

def document_tokenize_function(examples):
    return transformer_tokenizer(examples['document_plaintext'], padding="max_length")

def question_tokenize_function(examples):
    return transformer_tokenizer(examples['question_text'], padding="max_length")

def switch_doc_name1(examples):
    return {"document_input_ids": examples['input_ids']}

def switch_doc_name2(examples):
    return {"document_attention_mask": examples['attention_mask']}

def switch_ques_name1(examples):
    return {"question_input_ids": examples['input_ids']}

def switch_ques_name2(examples):
    return {"question_attention_mask": examples['attention_mask']}

def get_final_dataset(data_set):
    return data_set\
        .map(document_tokenize_function, batched=True, num_proc=4, remove_columns=["document_plaintext"])\
        .map(switch_doc_name1, batched=True, num_proc=4, remove_columns=["input_ids"])\
        .map(switch_doc_name2, batched=True, num_proc=4, remove_columns=["attention_mask"])\
        .map(question_tokenize_function, batched=True, num_proc=4, remove_columns=["question_text"])\
        .map(switch_ques_name1, batched=True, num_proc=4, remove_columns=["input_ids"])\
        .map(switch_ques_name2, batched=True, num_proc=4, remove_columns=["attention_mask"])


train_dl = get_final_dataset(english_label_train_set).remove_columns("token_type_ids")
val_dl = get_final_dataset(english_label_val_set).remove_columns("token_type_ids")


Using custom data configuration copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6
Reusing dataset parquet (/home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-dbe296a96306378b.arrow
Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-9c5553adc4f99418.arrow


Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-7804682ce4cfa673.arrow


Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-73c4a07fb746d0f8.arrow


...

Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-1eb7b1b8514d83e7.arrow


Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-7af38acfe5bffd57.arrow


Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-bf24248d1e3a002d.arrow


Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-a279d569ddfc314c.arrow


Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e97e83d5a2959e59.arrow


In [287]:
def collate_batch_bilstm(dataset):
    label = []
    document = []
    document_mask = []
    question = []
    question_mask = []
    for data in dataset:
        label.append(data["label"])
        document.append(data["document_input_ids"])
        document_mask.append(data["document_attention_mask"])
        question.append(data["question_input_ids"])
        question_mask.append(data["question_attention_mask"])

    label = torch.Tensor(label).cuda()
    document = torch.Tensor(document).cuda()
    document_mask = torch.tensor(document_mask).cuda()
    question = torch.tensor(question).cuda()
    question_mask = torch.tensor(question_mask).cuda()

    return document, document_mask, question, question_mask, label


train_dl = torch.utils.data.DataLoader(train_dl, batch_size=32, collate_fn=collate_batch_bilstm)
valid_dl = torch.utils.data.DataLoader(val_dl, batch_size=32, collate_fn=collate_batch_bilstm)

In [288]:
for batch in train_dl:
    print(batch)

(tensor([[  101., 25231.,  1768.,  ...,     0.,     0.,     0.],
        [  101.,  1109., 10412.,  ...,     0.,     0.,     0.],
        [  101., 12120.,  7531.,  ...,     0.,     0.,     0.],
        ...,
        [  101.,  3637., 20756.,  ...,     0.,     0.,     0.],
        [  101.,  1109.,  1148.,  ...,     0.,     0.,     0.],
        [  101., 10117.,  1867.,  ...,     0.,     0.,     0.]],
       device='cuda:0'), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), tensor([[ 101, 1332, 1108,  ...,    0,    0,    0],
        [ 101, 2627, 1108,  ...,    0,    0,    0],
        [ 101, 1332, 1110,  ...,    0,    0,    0],
        ...,
        [ 101, 2627, 1108,  ...,    0,    0,    0],
        [ 101, 1332, 1108,  ...,    0,    0,    0],
        [ 101, 1332, 1225,  ...,    0,    0,    0]], device='cuda:0

ValueError: expected sequence of length 512 at dim 1 (got 1023)

In [289]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
'''
@File    : preprocess.py.py
@IDE     : PyCharm
@Author  : Yaokun Li
@Date    : 2022/10/20 15:48
@Description :
'''
import torch
import numpy as np
from torch.utils.data import Dataset
from bpemb import BPEmb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer

device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")


dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"]
validation_set = dataset["validation"]

def getLanguageDataSet(data, language):
    def printAndL(x):
        return x["language"] == language
    return data.filter(printAndL)

def getEnglishDataSet(data):
    return getLanguageDataSet(data, "english")

# keep only english data
english_train_set = getEnglishDataSet(train_set)
english_validation_set = getEnglishDataSet(validation_set)

# delete useless data
english_train_set = english_train_set.remove_columns(["document_title", "language", "document_url"])
english_validation_set = english_validation_set.remove_columns(["document_title", "language", "document_url"])

def label_map_func(examples):
    labels = []
    for i in examples["annotations"]:
        if i["answer_start"] == [-1]:
            labels.append([0])

        else:
            labels.append([1])
    return {"label": labels}

english_label_train_set = english_train_set.map(label_map_func , batched=True, num_proc=4, remove_columns=["annotations"])
english_label_val_set = english_validation_set.map(label_map_func , batched=True, num_proc=4, remove_columns=["annotations"])

transformer_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

block_size = 128

def document_tokenize_function(examples):
    return transformer_tokenizer(examples['document_plaintext'], padding=True)

def question_tokenize_function(examples):
    return transformer_tokenizer(examples['question_text'], padding=True)

def switch_doc_name1(examples):
    return {"document_input_ids": examples['input_ids']}

def switch_doc_name2(examples):
    return {"document_attention_mask": examples['attention_mask']}

def switch_ques_name1(examples):
    return {"question_input_ids": examples['input_ids']}

def switch_ques_name2(examples):
    return {"question_attention_mask": examples['attention_mask']}

def get_final_dataset(data_set):
    return data_set\
        .map(document_tokenize_function, batched=True, num_proc=4, remove_columns=["document_plaintext"])\
        .map(switch_doc_name1, batched=True, num_proc=4, remove_columns=["input_ids"])\
        .map(switch_doc_name2, batched=True, num_proc=4, remove_columns=["attention_mask"])\
        .map(question_tokenize_function, batched=True, num_proc=4, remove_columns=["question_text"])\
        .map(switch_ques_name1, batched=True, num_proc=4, remove_columns=["input_ids"])\
        .map(switch_ques_name2, batched=True, num_proc=4, remove_columns=["attention_mask"])


train_dl = get_final_dataset(english_label_train_set).remove_columns("token_type_ids")
val_dl = get_final_dataset(english_label_val_set).remove_columns("token_type_ids")


def collate_batch_bilstm(dataset):
    label = []
    document = []
    document_mask = []
    question = []
    question_mask = []
    for data in dataset:
        label.append(data["label"])
        document.append(data["document_input_ids"])
        document_mask.append(data["document_attention_mask"])
        question.append(data["question_input_ids"])
        question_mask.append(data["question_attention_mask"])

    label = torch.Tensor(label).cuda()
    document = torch.Tensor(document).cuda()
    document_mask = torch.tensor(document_mask).cuda()
    question = torch.tensor(question).cuda()
    question_mask = torch.tensor(question_mask).cuda()

    return document, document_mask, question, question_mask, label


train_dl = torch.utils.data.DataLoader(train_dl, batch_size=32, collate_fn=collate_batch_bilstm)
valid_dl = torch.utils.data.DataLoader(val_dl, batch_size=32, collate_fn=collate_batch_bilstm)

for batch in train_dl:
    print(batch)

Using custom data configuration copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6
Reusing dataset parquet (/home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-dbe296a96306378b.arrow
Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-9c5553adc4f99418.arrow


Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-7804682ce4cfa673.arrow


Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-73c4a07fb746d0f8.arrow


...

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

(tensor([[  101., 25231.,  1768.,  ...,     0.,     0.,     0.],
        [  101.,  1109., 10412.,  ...,     0.,     0.,     0.],
        [  101., 12120.,  7531.,  ...,     0.,     0.,     0.],
        ...,
        [  101.,  3637., 20756.,  ...,     0.,     0.,     0.],
        [  101.,  1109.,  1148.,  ...,     0.,     0.,     0.],
        [  101., 10117.,  1867.,  ...,     0.,     0.,     0.]],
       device='cuda:0'), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), tensor([[  101,  1332,  1108,  9539,  1768,  2749,  1872,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [  101,  2627,  1108,  1103,  1148, 10412,  4716,  2981,  1111,  7594,
       

ValueError: expected sequence of length 1153 at dim 1 (got 526)